In [1]:
import numpy as np

In [2]:
with open("../data/day6.txt") as f:
    data = f.read()

arr = np.array([list(line) for line in data.split("\n")])

In [3]:
def change_direction(current_direction):
    if (current_direction == np.array([-1, 0])).all():
        return np.array([0, 1])
    elif (current_direction == np.array([0, 1])).all():
        return np.array([1, 0])
    elif (current_direction == np.array([1, 0])).all():
        return np.array([0, -1])
    elif (current_direction == np.array([0, -1])).all():
        return np.array([-1, 0])
    else:
        raise ValueError("Wrong direction")


def check_outside_bounds(test_position, size):
    return (
        test_position[0] == -1
        or test_position[1] == -1
        or test_position[0] == size[0]
        or test_position[1] == size[1]
    )

In [4]:
temp_arr = arr.copy()
walked_positions = []
current_position = np.asarray(np.where(arr == "^")).T[0]
direction = np.array([-1, 0])
while True:
    temp_arr[*current_position] = "X"
    walked_positions.append(tuple(current_position))

    test_position = current_position + direction
    if check_outside_bounds(test_position, arr.shape):
        break
    if temp_arr[*test_position] == "#":
        direction = change_direction(direction)
    current_position += direction

In [5]:
len(set(walked_positions))

4374

In [ ]:
def display_arr(arr):
    print("\n".join("".join(row) for row in arr))


# display_arr(temp_arr)

# Part 2

In [8]:
starting_position = np.asarray(np.where(arr == "^")).T[0]
starting_direction = np.array([-1, 0])

In [10]:
# A wall can only have effect if you would have gone there.
# We need only the unique positions
# We can not put a wall on the starting position
wall_options = list(dict.fromkeys(walked_positions))[1:]
len(set(wall_options))

4373

In [58]:
import tqdm

loops = 0

positions_full_loop = []


for y, x in tqdm.tqdm(wall_options):

    if (np.array([y, x]) == starting_position).all():
        continue

    temp_arr = arr.copy()

    temp_arr[y, x] = "O"


    states = []


    temp_arr[*starting_position] = "^"
    current_position = starting_position.copy()

    direction = starting_direction.copy()

    while True:

        state = (tuple(current_position), tuple(direction))

        states.append(state)

        test_position = current_position + direction

        if check_outside_bounds(test_position, temp_arr.shape):

            break

        if temp_arr[*test_position] in ("#", "O"):

            direction = change_direction(direction)


            test_position = current_position + direction
            if check_outside_bounds(test_position, temp_arr.shape):
                break
            if temp_arr[*test_position] in ("#", "O"):
                direction = change_direction(direction)

            # This can only happen twice - there is no way to run into a square that has walls on 4 sides
            # or on 3 sides from your direction (I suppose technically if it is on the bottom of the starting position)

        current_position += direction

        temp_arr[*current_position] = "X"


        if (tuple(current_position), tuple(direction)) in states:

            loops += 1

            positions_full_loop.append((x, y))

            break

100%|██████████| 4373/4373 [23:12<00:00,  3.14it/s]


In [59]:
print(loops)  # 1717 = too high
print(len(set(positions_full_loop)))

1705
1705


### Speed up

In [ ]:
# This should be faster, but it gives the wrong answer
# Find all states where you were
loops = 0
positions = []

temp_arr = arr.copy()

current_position = starting_position.copy()
temp_arr[*starting_position] = "^"
direction = starting_direction.copy()
states_original = []
while True:
    state = (tuple(current_position), tuple(direction))
    states_original.append(state)
    test_position = current_position + direction
    if check_outside_bounds(test_position, temp_arr.shape):
        break
    if temp_arr[*test_position] == "#":
        direction = change_direction(direction)
    current_position += direction
    temp_arr[*current_position] = "X"

placed = {state: i for i, state in enumerate(states_original)}
loops = 0
positions = []

for i, (pos, state_direction) in enumerate(tqdm.tqdm(states_original)):
    y, x = pos
    if (np.array([y, x]) == starting_position).all():
        continue
    temp_arr = arr.copy()
    temp_arr[y, x] = "O"

    states = states_original[:i]

    state = states[-1]
    current_position = state[0]
    temp_arr[*starting_position] = "^"
    direction = np.array(state[1])
    states = states_original[: i - 1]
    while True:
        state = (tuple(current_position), tuple(direction))
        try:
            where = placed[state]
        except KeyError:
            pass
        else:
            if where < i - 1:

                loops += 1
                positions.append((x, y))
                break
            if where > i:  # We're back on the further path
                break
        states.append(state)
        test_position = current_position + direction
        if check_outside_bounds(test_position, temp_arr.shape):
            break
        if temp_arr[*test_position] in ("#", "O"):
            direction = change_direction(direction)

            test_position = current_position + direction
            if check_outside_bounds(test_position, temp_arr.shape):
                break
            if temp_arr[*test_position] in ("#", "O"):
                direction = change_direction(direction)

        current_position += direction
        temp_arr[*current_position] = "X"

        if (tuple(current_position), tuple(direction)) in states:
            if i == 245:
                print("LOOP", where, i - 1)
            loops += 1
            positions.append((x, y))
            # display_arr(temp_arr)
            break

print(len(set(positions)))